<h1>logistic regression without applying polynomin</h1>
# this one should use less RAM than last version(v001)

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures


# df = pd.read_csv('./train.csv', index_col = True)
url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/train.csv'
df = pd.read_csv(url, index_col="id")
df['target'] = df['target'].astype(int)

parameters_dictionary = {}

for element in df:
    if element != "target":
        parameters_dictionary.update({element : df[element]})



label_dictionary = {'label': df['target']}

df_X = pd.DataFrame(data = parameters_dictionary)
df_Y = pd.DataFrame(data = label_dictionary)


# print(df_Y)

# print(df_Y.groupby('label').size())



In [31]:
# implement logistic regression


###
#   function: predict_logistic_regression
#       @param: input_x => training dataset features
#       @param: input_y => training dataset label
#       @param: input_z => testing dataset records (features only)
#   
#   output: array/list of predicted result of input_z
###
model = LogisticRegression(max_iter=10000)

def predict_logistic_regression(input_x, input_y, input_z):
    # build the model based on training data
    model.fit(input_x, input_y)

    # insert testing dataset to get the predicted output
    predicted_values = model.predict(input_z)

    return predicted_values
    


In [32]:
#  Applying 5-fold-cross-validation
k = 5
kf = KFold(n_splits = k, random_state = None)


# A function to run 5-fold cross validation on polynomial_logistic_regression
def kFold_logistic_regression(input_X, input_Y):

    validation_f1_score = []

    for train_index , validation_index in kf.split(input_X):

        # print('Start model fitting, k=', train_index[0], ' - ', train_index[-1], ' ...')

        X_train , X_valid = input_X.iloc[train_index,:], input_X.iloc[validation_index,:]
        y_train, y_valid = input_Y.iloc[train_index], input_Y.iloc[validation_index]

        # print('X_train: ', X_train)

        y_train = y_train.to_numpy().ravel()
        y_valid = y_valid.to_numpy().ravel()
        
        pred_testValues = predict_logistic_regression(X_train, y_train, X_valid)

        # count the f1 score(true records / predicted records)
        valid_f1 = f1_score(y_valid, pred_testValues)
        validation_f1_score.append(valid_f1)

    return validation_f1_score


In [35]:
testing_f1 = []

testing_f1_score = kFold_logistic_regression(df_X, df_Y)
testing_f1.append(testing_f1_score)


# print('testing set F1 score (poly 2 to 12) under 5-fold-cross-validation: ')
for row in testing_f1:
    print(row)

[0.8055555555555555, 0.8059701492537314, 0.7868852459016393, 0.71875, 0.7272727272727273]


<h3>Real prediction on testing data</h3>

In [ ]:
url_test = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/test.csv'
df_test = pd.read_csv(url_test, index_col="id")


